In [94]:
#Importem les llibreries necessàries:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Definim les variables:
URL_BASE = "https://www.habitaclia.com/alquiler-barcelona-"
#Camuflem l'user agent simulant ser un navegador de Mozilla:
headers = {'User-Agent': 'Mozilla/5.0'}
#Tenim un total de 929 pàgines d'anuncis...així que només extraurem les dades de les 100 primeres pàgines:
NUM_PAGS = 100
comptador = 0
llista=[]
Preus=[]
#Indiquem que apareixen 15 immobles per a cada pàgina (comptant el 0 com a primer valor):
Immobles_Per_Pagina=14

#Controlem les restriccions del fitxer robots.txt:
response = requests.get('https://www.habitaclia.com/robots.txt')
robots = response.text
print("\nFitxer robots.txt de https://www.habitaclia.com")
print("Verificació de restriccions:\n")
print(robots)
print("\nRealitzant scraping")

#Creem un bucle per a recórrer les pàgines d'habitaclia:
for i in range(1, NUM_PAGS):

    if i < 1:
        url = URL_BASE
    else:
        url = "%s%d.htm" % (URL_BASE, i)
        print(url)
        req = requests.get(url,headers=headers)
        
        #Comprovem que la petició retorna Status Code = 200:
        StatusCode = req.status_code
        if StatusCode == 200:
            
            #Passem el contingut HTML de la web a un objecte BeautifulSoup():
            html = BeautifulSoup(req.text, "html.parser")
            
            #Obtenim tots els divs de les entrades:
            panell = html.find(id="js-list")
            entrades = panell.find_all(class_="list-item-content")
            
            #Recorrem totes les entrades per a l'extracció de les dades d'interès:
            for entrada in entrades:
               
                #Barri de Barcelona on es troba l'immoble:
                Barri = entrada.find(class_ = 'list-item-location').get_text()
                
                #Tipus d'immoble de l'anunci:               
                Tipus_Immoble = entrada.find(class_ = 'list-item-title').get_text()
                             
                #Creem un bucle per a obtenir el preu de l'immoble:
                Tot_preus = panell.select(".list-item-content-second .font-2")
                Preus = [p.get_text() for p in Tot_preus]
              
                #Metres quadrats de l'immoble:
                Metres_Quadrats = entrada.find(class_= 'list-item-feature').get_text()
                
                #Nombre de lavabos de l'immoble:
                Num_Lavabos = entrada.find(class_= 'list-item-feature').get_text()
                
                #Nombre d'habitacions de l'immoble:
                Num_Habitacions = entrada.find(class_= 'list-item-feature').get_text()
                          
                #S'afegeixen els elements extrets a una llista:
                llista.append({
                       "Barri": Barri,
                       "Tipus_Immoble": Tipus_Immoble,
                       "Preu": Preus[comptador],
                       "Metres_Quadrats": Metres_Quadrats,
                       "Num_Lavabos": Num_Lavabos,
                       "Num_Habitacions": Num_Habitacions
                      })
                    
                #Convertim la llista en una DataFrame:
                df=pd.DataFrame(llista)
                
                #Extraiem el contingut necessari a través d'expressions regulars:
                df["Num_Habitacions"]=df.Num_Habitacions.str.extract('.+(\d+) habitaciones')
                df['Barri']=df.Barri.str.extract('Barcelona - (.+)')
                df["Tipus_Immoble"]=df.Tipus_Immoble.str.extract('Alquiler (.+?)\s')
                df['Metres_Quadrats']=df.Metres_Quadrats.str.extract('(\d+)')
                df['Num_Lavabos']=df.Num_Lavabos.str.extract('.+(\d+) baño')
                                   
                #Creem un comptador per a recórrer els 15 immobles que té cada pàgina:
                comptador += 1  
                if comptador > Immobles_Per_Pagina:
                   comptador = 0
                
        else:
        #Si no existeix la página, retorna "Error 400":
            break 

#Generem un fitxer csv a partir del DataFrame creat, indicant-li la codificació 'utf-16', que no tingui en compte el número
#de les files, que el fitxer té capçaleres i la separació entre les dades:
df.to_csv('Habitaclia_pisos_barcelona.csv', encoding='utf-16', index=False, header=True, sep='\t')


Fitxer robots.txt de https://www.habitaclia.com
Verificació de restriccions:

User-agent: *
Disallow: /hab_usuarios/registrocorreo.asp*
Disallow: /hab_usuarios/ajax/*
Disallow: /hab_inmuebles/ajax/*
Disallow: /dotnet/NotificacionesLiveListado/GetNotificacionesLiveListado*
Disallow: /dotnet/solicitud/vertelefono*
Disallow: /dotnet/solicitud/ValidarCaptcha*
Disallow: /dotnet/ficha/favrate*
Disallow: /dotnet/ficha/favcomment*
Disallow: /dotnet/ficha/translate*
Disallow: /*.txt$
Allow: /robots.txt
Allow: /app-ads.txt
Allow: /ads.txt

Realitzant scraping
https://www.habitaclia.com/alquiler-barcelona-1.htm
https://www.habitaclia.com/alquiler-barcelona-2.htm
https://www.habitaclia.com/alquiler-barcelona-3.htm
https://www.habitaclia.com/alquiler-barcelona-4.htm
https://www.habitaclia.com/alquiler-barcelona-5.htm
https://www.habitaclia.com/alquiler-barcelona-6.htm
https://www.habitaclia.com/alquiler-barcelona-7.htm
https://www.habitaclia.com/alquiler-barcelona-8.htm
https://www.habitaclia.com/